In this notebook, I explore the data using regression analysis

In [1]:
import pandas as pd
import scipy.sparse as ss
import statsmodels.api as sm

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

In [2]:
meta = pd.read_pickle('files/creep.pkl')
meta

,title,date,subgenre,rating,time,author,polarity,subjectivity,title_length,subgenre_count,num_words,mean_word_freq,num_unique_words,unique_ratio
0,blood magic,2020-07-24,beings entities monsters creatures cryptids ...,8.15,8,Tobias Wade,0.046569,0.501569,2,10,966,0.001789,559,0.578675
1,a diner open 25 hours a day,2020-07-23,abductions kidnappings beings entities scien...,8.08,12,Christopher Maxim,0.033834,0.514034,7,7,1369,0.001193,838,0.612126
2,a shattered life,2020-07-22,madness paranoia mental illness monsters crea...,9.10,14,Matt Dymerski,0.054970,0.531194,3,10,1496,0.001259,794,0.530749
3,always be nice to your neighbors,2020-07-21,deaths murders disappearances,7.45,4,Christine Druga,0.020018,0.402989,6,3,350,0.004367,229,0.654286
4,vantablack a death metal cult,2020-07-20,rites rituals,7.63,13,Christopher Maxim,0.120475,0.501398,5,2,1338,0.001477,677,0.505979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3494,the corner,2008-03-23,locations sites,6.85,2,Author Unknown,0.047026,0.527637,2,2,121,0.009709,103,0.851240
3495,the gallery of henri beauchamp,2008-03-22,rites rituals,8.28,5,Author Unknown,0.148358,0.508518,5,2,543,0.002786,359,0.661142
3496,the grove,2008-03-21,locations sites,7.67,1,Author Unknown,0.343233,0.539098,2,2,89,0.016949,59,0.662921
3497,the abandoned convenience store,2008-03-20,rites rituals,7.49,1,Author Unknown,-0.027165,0.549320,4,2,106,0.011628,86,0.811321


First, regress ratings onto the other variables.

In [3]:
X = meta.loc[:,'polarity':'unique_ratio'].join(meta['time'])
y = meta['rating']

# standard scaling for model
X = (X-X.mean())/X.std()

In [4]:
mod = sm.OLS(y,X).fit()
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 rating   R-squared (uncentered):                   0.003
Model:                            OLS   Adj. R-squared (uncentered):              0.000
Method:                 Least Squares   F-statistic:                              1.059
Date:                Tue, 03 Nov 2020   Prob (F-statistic):                       0.390
Time:                        17:14:45   Log-Likelihood:                         -12083.
No. Observations:                3499   AIC:                                  2.418e+04
Df Residuals:                    3490   BIC:                                  2.424e+04
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
polarity             0.0006      0.130      0.005      0.996      -0.255       0.256
subjectivity        -0.0394      0.130     -0.304      0.761      -0.294       0.215
title_length         0.0624      0.130      0.478      0.633      -0.193       0.318
subgenre_count      -0.0073      0.132     -0.055      0.956      -0.266       0.251
num_words           -0.0017      0.136     -0.013      0.990      -0.268       0.265
mean_word_freq      -0.0787      0.164     -0.479      0.632      -0.401       0.243
num_unique_words     0.2074      0.380      0.545      0.586      -0.538       0.953
unique_ratio        -0.2653      0.204     -1.298      0.194      -0.666       0.135
time                -0.1256      0.360     -0.349      0.727      -0.831       0.580
==============================================================================
Omnibus:                      582.024   Durbin-Watson:                   0.037
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1065.565
Skew:                          -1.043   Prob(JB):                    4.13e-232
Kurtosis:                       4.720   Cond. No.                         6.82
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The collection of story variables explains virtually none of the variance in the ratings, and none of the variables are statistically significant in this ensemble.

However, there may be significant multicollinearity problems with using all available predictors in multiple regression. Using a subset of variables that are most strongly correlated with rating in a regression will cull the number of predictors.

In [5]:
# square to eliminate negatives and accentuate the signal
meta.corr().loc['rating']**2

rating              1.000000
time                0.057003
polarity            0.000193
subjectivity        0.000965
title_length        0.006230
subgenre_count      0.003101
num_words           0.006345
mean_word_freq      0.061400
num_unique_words    0.079219
unique_ratio        0.111508
Name: rating, dtype: float64

The most correlated variables with rating are the reading time, the mean word frequency, the number of unique words in the story, and the ratio of unique words to total words in the story. Since unique_ratio and num_unique_words essentially measure the same thing, eliminate the raw number of unique words to reduce multicollinearity.

In [6]:
# subset and standardize
X = meta[['time', 'mean_word_freq', 'unique_ratio']]
X = (X-X.mean())/X.std()

In [7]:
mod = sm.OLS(y,X).fit()
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 rating   R-squared (uncentered):                   0.003
Model:                            OLS   Adj. R-squared (uncentered):              0.002
Method:                 Least Squares   F-statistic:                              2.978
Date:                Tue, 03 Nov 2020   Prob (F-statistic):                      0.0303
Time:                        17:14:49   Log-Likelihood:                         -12083.
No. Observations:                3499   AIC:                                  2.417e+04
Df Residuals:                    3496   BIC:                                  2.419e+04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
time               0.0404      0.175      0.230      0.818      -0.304       0.385
mean_word_freq    -0.1084      0.155     -0.699      0.484      -0.412       0.195
unique_ratio      -0.2890      0.199     -1.452      0.147      -0.679       0.101
==============================================================================
Omnibus:                      588.993   Durbin-Watson:                   0.037
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1080.881
Skew:                          -1.053   Prob(JB):                    1.95e-235
Kurtosis:                       4.726   Cond. No.                         2.75
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Regression using this subset of predictors yields essentially the same results as before. Now, examine the word frequencies themselves as predictors and see if they have explanatory power. 

In [8]:
stories = pd.read_pickle('files/corpus.pkl')

In [9]:
# remove common noise words
extras = ['like', 'just', 'said', 'im', 'didnt', 'dont', 'did', 'youre', 'youare', 'werent']
stop_words = text.ENGLISH_STOP_WORDS.union(extras)

In [10]:
# create sparse document-term matrix 
vectorizer = TfidfVectorizer(stop_words=stop_words, max_df=0.8) # upper bound to reduce noise terms
dtm = vectorizer.fit_transform(stories['story'])
dtm

<3499x62947 sparse matrix of type '<class 'numpy.float64'>'
	with 1833609 stored elements in Compressed Sparse Row format>

Since there are over 60,000 predictors, it is helpful to reduce the dimensions of the document-term matrix using SVD to isolate a signal. 

In [11]:
# dimension reduction using singular values 
svd = TruncatedSVD(n_components=50)
normalizer = Normalizer(copy=False)
pca = make_pipeline(svd, normalizer)
U = pca.fit_transform(dtm)
y = meta['rating']

The first 50 singular vectors explan around 11% of the variation in the original dtm:

In [12]:
svd.explained_variance_ratio_.sum()

0.11150732411989929

Now, regress the ratings onto these 50 principle components:

In [13]:
U = (U-U.mean())/U.std()
mod = sm.OLS(y,U).fit()
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 rating   R-squared (uncentered):                   0.955
Model:                            OLS   Adj. R-squared (uncentered):              0.954
Method:                 Least Squares   F-statistic:                              1465.
Date:                Tue, 03 Nov 2020   Prob (F-statistic):                        0.00
Time:                        17:16:35   Log-Likelihood:                         -6660.8
No. Observations:                3499   AIC:                                  1.342e+04
Df Residuals:                    3449   BIC:                                  1.373e+04
Df Model:                          50                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.6543      0.008    196.086      0.000       1.638       1.671
x2            -0.0370      0.021     -1.740      0.082      -0.079       0.005
x3            -0.3676      0.021    -17.316      0.000      -0.409      -0.326
x4            -0.1368      0.026     -5.290      0.000      -0.188      -0.086
x5             0.0360      0.026      1.387      0.165      -0.015       0.087
x6            -0.1286      0.028     -4.571      0.000      -0.184      -0.073
x7            -0.1592      0.028     -5.713      0.000      -0.214      -0.105
x8             0.1318      0.033      3.977      0.000       0.067       0.197
x9             0.3874      0.034     11.534      0.000       0.322       0.453
x10            0.4796      0.034     14.027      0.000       0.413       0.547
x11            0.1005      0.036      2.820      0.005       0.031       0.170
x12            0.3209      0.050      6.476      0.000       0.224       0.418
x13            0.2828      0.042      6.765      0.000       0.201       0.365
x14            0.3399      0.037      9.085      0.000       0.267       0.413
x15           -0.5016      0.040    -12.610      0.000      -0.580      -0.424
x16            0.3970      0.049      8.031      0.000       0.300       0.494
x17            0.5832      0.048     12.117      0.000       0.489       0.678
x18            0.1153      0.048      2.385      0.017       0.021       0.210
x19            0.4305      0.045      9.470      0.000       0.341       0.520
x20           -0.2372      0.051     -4.624      0.000      -0.338      -0.137
x21           -0.2562      0.046     -5.607      0.000      -0.346      -0.167
x22            0.0118      0.044      0.267      0.790      -0.075       0.099
x23            0.0418      0.046      0.903      0.366      -0.049       0.133
x24           -0.1640      0.044     -3.751      0.000      -0.250      -0.078
x25           -0.0835      0.044     -1.902      0.057      -0.170       0.003
x26           -0.1053      0.046     -2.288      0.022      -0.195      -0.015
x27           -0.0280      0.042     -0.675      0.499      -0.109       0.053
x28            0.1338      0.041      3.272      0.001       0.054       0.214
x29            0.5026      0.043     11.617      0.000       0.418       0.587
x30            0.2022      0.045      4.541      0.000       0.115       0.290
x31           -0.0952      0.041     -2.299      0.022      -0.176      -0.014
x32           -0.1618      0.044     -3.704      0.000      -0.248      -0.076
x33            0.2691      0.048      5.609      0.000       0.175       0.363
x34           -0.1060      0.043     -2.454      0.014      -0.191      -0.021
x35           -0.3293      0.046     -7.210      0.000      -

Amazingly, by only capturing 11% of the variance in word frequencies, over 90% of the variance in rating is explained by the first principal component:

In [14]:
mod = sm.OLS(y,U[:, 0]).fit()
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 rating   R-squared (uncentered):                   0.933
Model:                            OLS   Adj. R-squared (uncentered):              0.933
Method:                 Least Squares   F-statistic:                          4.885e+04
Date:                Tue, 03 Nov 2020   Prob (F-statistic):                        0.00
Time:                        17:16:38   Log-Likelihood:                         -7353.7
No. Observations:                3499   AIC:                                  1.471e+04
Df Residuals:                    3498   BIC:                                  1.472e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.5869      0.007    221.025      0.000       1.573       1.601
==============================================================================
Omnibus:                      139.047   Durbin-Watson:                   1.814
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              167.515
Skew:                           0.448   Prob(JB):                     4.21e-37
Kurtosis:                       3.589   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Conclusions:

While the above results are promising, they must be contextualized. The >90% predictor is the 1st principal component of the SVD of the orginal document-term matirx. This is a linear combination of some word frequencies across the documents given by the TF-IDF vectorizer. Essentially, >90% of the variance is explained by some abstruse combination of word frequencies. It would be vary difficult to identify which words these are and in what order/combination they occur in a story based on just the SVD of TF-IDF frequencies. Nevertheless, it may be possible to construct a model that captures this word frequency variance and predicts rating. 